In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

#---------------------------------------------------------------------
import sys, os
import re
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import random

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutagePredictor import OutagePredictor
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
#---------------------------------------------------------------------

In [ ]:
cbus_only = True

In [ ]:
if cbus_only:
    trsf_pole_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\20230401_20240831\Outages\trsf_location_info_df.pkl')
    trsf_pole_nbs_to_ignore = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']
    #-------------------------
    cbus_zips = [
        43085, 
        43201, 
        43202, 
        43203, 
        43204, 
        43205, 
        43206, 
        43207, 
        43209, 
        43210, 
        43211, 
        43212, 
        43213, 
        43214, 
        43215, 
        43216, 
        43217, 
        43218, 
        43219, 
        43220, 
        43221, 
        43222, 
        43223, 
        43224, 
        43226, 
        43227, 
        43228, 
        43229, 
        43230, 
        43231, 
        43232, 
        43234, 
        43235, 
        43236, 
        43240
    ] 
    
    #----------------------------------------------------------------------------------------------------
    trsf_pole_df = trsf_pole_df.drop_duplicates()
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    trsf_pole_df = trsf_pole_df[~trsf_pole_df['trsf_pole_nb'].isin(trsf_pole_nbs_to_ignore)]
    #-------------------------
    # Want only one entry per zip
    # For each trsf_pole_nb, keep only the entry with the largest nPNs value
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    trsf_pole_df = trsf_pole_df.loc[trsf_pole_df.groupby('trsf_pole_nb')['nPNs_xfmri_sa4i'].idxmax()]
    trsf_pole_df = trsf_pole_df.reset_index(drop=True)
    assert(trsf_pole_df.shape[0]==trsf_pole_df['trsf_pole_nb'].nunique())
    #-------------------------
    trsf_pole_df['zip'] = trsf_pole_df['zip'].astype(int)
    trsf_pole_df_cbus = trsf_pole_df[trsf_pole_df['zip'].isin(cbus_zips)]

In [ ]:
model_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10'

In [ ]:
prediction_date   = pd.to_datetime('2023-07-27')

In [ ]:
tmp_f = open(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20240906\Models\All_EEMSP_agg_Top10\Slices\ledger.json')
ledger = json.load(tmp_f)
tmp_f.close()
trsf_pole_nbs_model = natsorted(ledger.keys())

In [ ]:
random.seed(42)

In [ ]:
n_poles = 100
if cbus_only:
    trsf_pole_nbs_init = list(set(trsf_pole_df_cbus['trsf_pole_nb'].unique()).intersection(set(trsf_pole_nbs_model)))
    trsf_pole_nbs_init = random.sample(trsf_pole_nbs_init,  n_poles)
else:
    trsf_pole_nbs_init = random.sample(trsf_pole_nbs_model, n_poles)

In [ ]:
self = OutagePredictor(prediction_date=prediction_date)
#-----
self.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v5')
#-----
self.set_trsf_pole_nbs(trsf_pole_nbs = trsf_pole_nbs_init)
print(f"n_trsf_pole_nbs = {len(self.trsf_pole_nbs)}")
print(f"\nSQL statement:\n{self.trsf_pole_nbs_sql}")
#-----
self.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)

In [ ]:
self.predict()
y_scores_df = self.y_pred_df.copy()

In [ ]:
y_scores_df['pred'].sum()

In [ ]:
y_scores_df = pd.merge(
    y_scores_df, 
    self.rcpx_df['EEMSP_0'], 
    how         = 'inner', 
    left_index  = True, 
    right_index = True
)
y_scores_df = y_scores_df.sort_values(by=[1], ascending=False)

In [ ]:
trsf_pole_nbs = self.trsf_pole_nbs

In [ ]:
ledger_inv_srs = OutagePredictor.group_trsf_pole_nbs_by_model(
    trsf_pole_nbs = self.trsf_pole_nbs, 
    ledger        = ledger
)

In [ ]:
# model_i = 7
model_i = 0
trsf_poles_i = ledger_inv_srs[model_i]

print(model_i)
model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
assert(os.path.isdir(model_i_dir))
#-------------------------
# If no model able to be made for group, directory will be empty
# if not os.listdir(model_i_dir):
#     print('No model compiled.\nContinuing to next')
#     continue
#-------------------------
predictor_i = OutagePredictor(prediction_date=prediction_date)
predictor_i.set_model_dir(model_i_dir)
predictor_i.set_trsf_pole_nbs(trsf_poles_i)
predictor_i.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)
#-------------------------
# assert(model_i not in predictors.keys())
# predictors[model_i] = predictor_i

In [ ]:
len(list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique()))))

In [ ]:
list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique())))

In [ ]:
dovs_df_new = self.build_dovs_df(
    mp_df             = None, 
    dt_0              = None, 
    dt_1              = None, 
    overlap_assertion = 0.95
)

In [ ]:
mp_df_new = self.build_active_mp_df(
    dt_0              = None, 
    dt_1              = None, 
)

In [ ]:
dovs_df_new_2 = self.build_dovs_df(
    mp_df             = mp_df_new, 
    dt_0              = None, 
    dt_1              = None, 
    overlap_assertion = 0.95
)

In [ ]:
df_mp_install_time_col = 'inst_ts'
df_mp_removal_time_col = 'rmvl_ts'
# dt_0 = prediction_date
# dt_1 = prediction_date
dt_0 = self.date_range[0]
dt_1 = self.date_range[1]

In [ ]:
mp_df = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
    trsf_pole_nbs               = trsf_pole_nbs, 
    join_curr_hist              = True, 
    addtnl_mp_df_curr_cols      = None, 
    addtnl_mp_df_hist_cols      = None, 
    assume_one_xfmr_per_PN      = True, 
    drop_approx_duplicates      = True, 
    drop_approx_duplicates_args = None, 
    df_mp_serial_number_col     = 'mfr_devc_ser_nbr', 
    df_mp_prem_nb_col           = 'prem_nb', 
    df_mp_install_time_col      = 'inst_ts', 
    df_mp_removal_time_col      = 'rmvl_ts', 
    df_mp_trsf_pole_nb_col      = 'trsf_pole_nb'
)

# Only want meters active at the relevant time period
mp_df = mp_df[(mp_df[df_mp_install_time_col]<=pd.to_datetime(dt_0)) & 
              (mp_df[df_mp_removal_time_col].fillna(pd.Timestamp.max)>pd.to_datetime(dt_1))]

In [ ]:
set(self.trsf_pole_nbs).difference(set(mp_df['trsf_pole_nb'].unique()))

In [ ]:
# Build dovs_df
dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        premise_nbs=mp_df['prem_nb'].unique().tolist(), 
        date_range=[dt_0, dt_1], 
        field_to_split='premise_nbs', 
        include_premise=True
    ), 
    build_consolidated=False
)
dovs_df = dovs.df.copy()

In [ ]:
dovs_df = pd.merge(
    dovs_df, 
    mp_df[['prem_nb', 'trsf_pole_nb']].drop_duplicates(), 
    left_on='PREMISE_NB', 
    right_on='prem_nb', 
    how='left'
)
dovs_df

In [ ]:
dovs_df_new.equals(dovs_df)

In [ ]:
print(dovs_df.shape)
print(dovs_df_new.shape)

In [ ]:
dovs_df.sort_values(by=['OUTG_REC_NB', 'trsf_pole_nb', 'PREMISE_NB'], ignore_index=True).equals(
dovs_df_new.sort_values(by=['OUTG_REC_NB', 'trsf_pole_nb', 'PREMISE_NB'], ignore_index=True)
)

In [ ]:
dovs_df.sort_values(by=['OUTG_REC_NB', 'trsf_pole_nb', 'PREMISE_NB'], ignore_index=True).equals(
dovs_df_new_2.sort_values(by=['OUTG_REC_NB', 'trsf_pole_nb', 'PREMISE_NB'], ignore_index=True)
)

In [ ]:
print(mp_df.shape)
print(mp_df_new.shape)

In [ ]:
mp_df.sort_values(by=['trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr'], ignore_index=True).equals(
mp_df_new.sort_values(by=['trsf_pole_nb', 'prem_nb', 'mfr_devc_ser_nbr'], ignore_index=True)
)

In [ ]:
dovs_df['trsf_pole_nb'].nunique()

In [ ]:
y_scores_df[y_scores_df['pred']==1].index.nunique()

In [ ]:
set(dovs_df['trsf_pole_nb'].unique()).intersection(set(y_scores_df[y_scores_df['pred']==1].index.unique()))

In [ ]:
self = predictor_i

# date          = '2023-07-26'
# date = (self.prediction_date - 0.5*(self.idk_name_1+self.idk_name_2)).date()
date = self.prediction_date - self.idk_name_2
evsSum_sql_kwargs = dict(opco='oh')
date_col='aep_event_dt'

In [ ]:
prediction_dates = pd.date_range(
    start = pd.to_datetime('2023-09-01'), 
    end   = pd.to_datetime('2023-09-03'), 
    freq  = pd.Timedelta('1D')
)
prediction_dates

In [ ]:
outg_prdctr = OutagePredictor(prediction_date=prediction_dates[0])
#-----
outg_prdctr.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v5')
outg_prdctr.set_trsf_pole_nbs(trsf_pole_nbs[:10])

outg_prdctr.prep_multiple_prediction_dates(
    prediction_dates  = prediction_dates, 
    predictions_range = None, 
    evsSum_sql_fcn    = AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs = dict(opco='oh'), 
    save_args         = False
)

In [ ]:
outg_prdctr.date_range

In [ ]:
outg_prdctr.daq_date_ranges

In [ ]:
outg_prdctr.evsSum_df['trsf_pole_nb'].nunique()

In [ ]:
len(trsf_pole_nbs)

In [ ]:
date_col='aep_event_dt'
trsf_pole_nb_col  = 'trsf_pole_nb'
self = outg_prdctr

In [ ]:
assert(0)

In [ ]:
predictors = dict()
for model_i, trsf_poles_i in ledger_inv_srs.items():
    print(model_i)
    model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
    assert(os.path.isdir(model_i_dir))
    #-------------------------
    # If no model able to be made for group, directory will be empty
    if not os.listdir(model_i_dir):
        print('No model compiled.\nContinuing to next')
        continue
    #-------------------------
    predictor_i = OutagePredictor(prediction_date=prediction_date)
    predictor_i.set_model_dir(model_i_dir)
    predictor_i.set_trsf_pole_nbs(trsf_poles_i)
    predictor_i.initialize_data(
        evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
        evsSum_sql_kwargs=dict(opco='oh'), 
    )
    #-------------------------
    assert(model_i not in predictors.keys())
    predictors[model_i] = predictor_i

In [ ]:
for model_i, pred_i in predictors.items():
    y_pred_i = pred_i.model_clf.predict(pred_i.X_test)
    print(model_i)
    print(y_pred_i.sum())
    print()

In [ ]:
pred_0 = predictors[0]
y_pred_0 = pred_0.model_clf.predict(pred_0.X_test)

In [ ]:
y_pred_0.sum()

In [ ]:
y_pred_0

In [ ]:
pred_0.rcpx_df

In [ ]:
conn_outages = Utilities.get_utldb01p_oracle_connection()

In [ ]:
sql_outage_location_ids = DOVSOutages_SQL.build_sql_find_outage_xfmrs(
    mjr_mnr_cause   = None, 
    include_premise = False, 
    date_range      = [dt_0, dt_1], 
    states          = ['OH']
)
sql_stmnt_outage_location_ids = sql_outage_location_ids.get_sql_statement()
#-------------------------
df_outage_location_ids = pd.read_sql_query(
    sql_stmnt_outage_location_ids, 
    conn_outages
)

In [ ]:
df_outage_location_ids

In [ ]:
#FUCK

In [ ]:
assert(0)

In [ ]:
trsf_pole_nb_col               = 'trsf_pole_nb'
t_min_col                      = 't_min'
t_max_col                      = 't_max'
trsf_pole_nbs_to_ignore        = [' ', 'TRANSMISSION', 'PRIMARY', 'NETWORK']
include_zero_counts            = True
build_sql_outage_kwargs        = None
assert_all_trsf_pole_nbs_found = False
save_args                      = False
batch_size                     = 30
n_update                       = 1
verbose                        = False

In [ ]:
# df          = pd.DataFrame(data=trsf_pole_nbs, columns=['trsf_pole_nb'])
# df['t_min'] = date_0
# df['t_max'] = date_1

prediction_date = pd.to_datetime('2024-07-27')
search_times_df = pd.DataFrame(
    data    = trsf_pole_df_cbus['trsf_pole_nb'].unique().tolist(), 
    columns = ['trsf_pole_nb']
)
search_times_df['t_min'] = prediction_date - pd.Timedelta('365D')
search_times_df['t_max'] = prediction_date - pd.Timedelta('1D')

In [ ]:
# search_times_df = search_times_df.iloc[:1000]
# search_times_df = search_times_df.iloc[:10]

In [ ]:
# n_outgs_per_xfmr_v1, n_outgs_per_xfmr_df_v1 = DOVSOutages.get_n_outgs_per_xfmr(
#     date_0                  = prediction_date - pd.Timedelta('365D'), 
#     date_1                  = prediction_date - pd.Timedelta('1D'), 
#     trsf_pole_nbs           = search_times_df['trsf_pole_nb'].unique().tolist(), 
#     include_zero_counts     = True, 
#     build_sql_outage_kwargs = None, 
#     save_args               = False, 
#     verbose                 = False, 
#     return_df               = True
# )

# n_outgs_per_xfmr_v1.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1.pkl')
# n_outgs_per_xfmr_df_v1.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1.pkl')

In [ ]:
# n_outgs_per_xfmr_v3, n_outgs_per_xfmr_df_v3 = DOVSOutages.get_n_outgs_per_xfmr_in_df(
#     search_times_df                = search_times_df, 
#     trsf_pole_nb_col               = trsf_pole_nb_col, 
#     t_min_col                      = t_min_col, 
#     t_max_col                      = t_max_col, 
#     trsf_pole_nbs_to_ignore        = trsf_pole_nbs_to_ignore, 
#     include_zero_counts            = True, 
#     build_sql_outage_kwargs        = build_sql_outage_kwargs, 
#     assert_all_trsf_pole_nbs_found = assert_all_trsf_pole_nbs_found, 
#     save_args                      = save_args, 
#     batch_size                     = batch_size, 
#     n_update                       = n_update, 
#     verbose                        = verbose, 
#     return_df                      = True
# )

# n_outgs_per_xfmr_v3.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3.pkl')
# n_outgs_per_xfmr_df_v3.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3.pkl')

In [ ]:
n_outgs_per_xfmr_v1 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1.pkl')
n_outgs_per_xfmr_df_v1 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1.pkl')

n_outgs_per_xfmr_v3 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3.pkl')
n_outgs_per_xfmr_df_v3 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3.pkl')

In [ ]:
n_outgs_per_xfmr_v3.droplevel(level=['t_min', 't_max']).equals(n_outgs_per_xfmr_v1)

In [ ]:
n_outgs_per_xfmr_v1.isna().sum()

In [ ]:
(n_outgs_per_xfmr_v1==0).sum()

In [ ]:
# n_outgs_per_xfmr_v1_wo0, n_outgs_per_xfmr_df_v1_wo0 = DOVSOutages.get_n_outgs_per_xfmr(
#     date_0                  = prediction_date - pd.Timedelta('365D'), 
#     date_1                  = prediction_date - pd.Timedelta('1D'), 
#     trsf_pole_nbs           = search_times_df['trsf_pole_nb'].unique().tolist(), 
#     include_zero_counts     = False, 
#     build_sql_outage_kwargs = None, 
#     save_args               = False, 
#     verbose                 = False, 
#     return_df               = True
# )

# n_outgs_per_xfmr_v1_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1_wo0.pkl')
# n_outgs_per_xfmr_df_v1_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1_wo0.pkl')

In [ ]:
# n_outgs_per_xfmr_v3_wo0, n_outgs_per_xfmr_df_v3_wo0 = DOVSOutages.get_n_outgs_per_xfmr_in_df(
#     search_times_df                = search_times_df, 
#     trsf_pole_nb_col               = trsf_pole_nb_col, 
#     t_min_col                      = t_min_col, 
#     t_max_col                      = t_max_col, 
#     trsf_pole_nbs_to_ignore        = trsf_pole_nbs_to_ignore, 
#     include_zero_counts            = False, 
#     build_sql_outage_kwargs        = build_sql_outage_kwargs, 
#     assert_all_trsf_pole_nbs_found = assert_all_trsf_pole_nbs_found, 
#     save_args                      = save_args, 
#     batch_size                     = batch_size, 
#     n_update                       = n_update, 
#     verbose                        = verbose, 
#     return_df                      = True
# )

# n_outgs_per_xfmr_v3_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3_wo0.pkl')
# n_outgs_per_xfmr_df_v3_wo0.to_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3_wo0.pkl')

In [ ]:
n_outgs_per_xfmr_v1_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v1_wo0.pkl')
n_outgs_per_xfmr_df_v1_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v1_wo0.pkl')

n_outgs_per_xfmr_v3_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_v3_wo0.pkl')
n_outgs_per_xfmr_df_v3_wo0 = pd.read_pickle(r'C:\Users\s346557\Downloads\n_outgs_per_xfmr_df_v3_wo0.pkl')

In [ ]:
n_outgs_per_xfmr_v3_wo0.droplevel(level=['t_min', 't_max']).sort_index().equals(n_outgs_per_xfmr_v1_wo0.sort_index())

In [ ]:
n_outgs_per_xfmr_v1_wo0.isna().sum()

In [ ]:
(n_outgs_per_xfmr_v1_wo0==0).sum()